In [9]:
# ---------------------------
# 1. Instalar dependencias
# ---------------------------
!pip install ollama langchain faiss-cpu ragas datasets sentence-transformers nest_asyncio
!pip install llama-index
!pip install llama-index-llms-ollama

from datasets import load_dataset
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from llama_index.llms.ollama import Ollama
from ragas.metrics import ContextPrecision, ContextRecall, Faithfulness, AnswerRelevancy
from ragas.evaluation import evaluate
from IPython.display import display, Markdown
import pandas as pd
import nest_asyncio
import asyncio

nest_asyncio.apply()

# Habilitar loop para Jupyter
nest_asyncio.apply()

# ---------------------------
# 2. Configurar LLM con Ollama
# ---------------------------
llm = Ollama(model="llama2")  # Puedes usar mistral o codellama si lo tienes descargado

# ---------------------------
# 3. Configurar embeddings locales
# ---------------------------
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# ---------------------------
# 4. Crear vector store con FAISS (simulación con contenido de ejemplo)
# ---------------------------
texts = [
    "La fotosíntesis es el proceso mediante el cual las plantas producen energía.",
    "La capital de Chile es Santiago.",
    "El sistema solar está compuesto por planetas que orbitan el Sol."
]

faiss_index = FAISS.from_texts(texts, embedding=embeddings)

# ---------------------------
# 5. Cargar dataset SQuAD y generar ejemplos
# ---------------------------
squad = load_dataset("squad", split="validation[:3]")  # Tomamos solo 3 ejemplos para demo

df_correctas = pd.DataFrame({
    "question": squad["question"],
    "answer": [a["text"][0] for a in squad["answers"]],
    "ground_truth": [a["text"][0] for a in squad["answers"]],
    "contexts": [["La capital de Chile es Santiago."]] * 3
})

df_incorrectas = df_correctas.copy()
df_incorrectas["answer"] = ["respuesta totalmente incorrecta"] * 3

df_parciales = df_correctas.copy()
df_parciales["answer"] = ["respuesta parcialmente correcta"] * 3

# ---------------------------
# 6. Configurar métricas RAGAS
# ---------------------------
context_precision = ContextPrecision()
context_recall = ContextRecall()
faithfulness = Faithfulness()
answer_relevancy = AnswerRelevancy()

# ---------------------------
# 7. Función para evaluar datasets
# ---------------------------
async def evaluar_dataset(df):
    resultados = await evaluate(
        df,
        metrics=[context_precision, context_recall, faithfulness, answer_relevancy],
        llm=llm,
        embeddings=embeddings  # ✅ Evita OpenAI
    )
    return resultados

# ---------------------------
# 8. Ejecutar evaluaciones
# ---------------------------
display(Markdown("### 🔹 Calculando métricas para respuestas CORRECTAS..."))
res_correctas = await evaluar_dataset(df_correctas)
display(res_correctas)

display(Markdown("### 🔹 Calculando métricas para respuestas INCORRECTAS..."))
res_incorrectas = await evaluar_dataset(df_incorrectas)
display(res_incorrectas)

display(Markdown("### 🔹 Calculando métricas para respuestas PARCIALES..."))
res_parciales = await evaluar_dataset(df_parciales)
display(res_parciales)


ImportError: cannot import name 'InvalidSchemaError' from 'pydantic._internal._generate_schema' (C:\Users\bakug\anaconda3\Lib\site-packages\pydantic\_internal\_generate_schema.py)